In [34]:
import xml.etree.ElementTree as ET
import pandas as pd
from langdetect import detect
import os
import re
from liste_auteurs import list_aut_str
from lxml import etree
import bs4 as bs

In [24]:
!pip install beautifulsoup4

In [2]:
list_files = os.listdir()
list_txt_files = []

for i in list_files:
    if i.endswith(".txt"):
        list_txt_files.append(i)

In [3]:
for file in list_txt_files:
    with open(file, 'r') as f :
        t = f.readlines()
    t = ''.join(t)
    t = re.sub(r"-\n", r"t", t)
    t = re.sub(r"\n", r" ", t)
    t = re.sub(r" <section> ", r"\n", t)
    t = re.sub(r"<section> ", r"", t)
    with open(file, 'w') as f :
        f.write(t)

In [4]:
for file in list_txt_files:
    with open(file, "r") as f:
        text = f.readlines()
    lang = []
    for i in text:
        l = detect(i)
        if l == 'ro' or l == 'ca':
            lang.append('co')
        elif l == 'it':
            test = i.split()
            count_u = test.count('u')
            count_il = test.count('il')
            if count_il > count_u:
                lang.append(l)
            else:
                lang.append('co')
        else:
            lang.append(l)
    
    
    aut = []
    for t in text:
        t = re.sub(r"’", r"'",t)
        dic = {}
        for i in list_aut_str:
            if i in t[-70:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            aut.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            aut.append('Inconnu')

    typ = []
    list_rub = ['art','let','poe','div','the','cha','feu','his']
    for t in text:
        dic = {}
        for i in list_rub:
            if i in t[-5:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            typ.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            typ.append('Non défini')
            
    df = pd.DataFrame([text,lang,aut,typ]).T
    df.columns = ['texte','langue','auteur','type']
    xml = df.to_xml()
    myre = '(bpt\w{7,10})'
    if re.search(myre, file):
        nfile = re.search(myre, file).group(1)
    with open(nfile + '.xml', "w+") as f:
        f.write(xml)

In [5]:
list_files = os.listdir()
list_xml_files = []

for i in list_files:
    if i.endswith(".xml"):
        list_xml_files.append(i)

In [6]:
for file in list_xml_files:
    xml_file = ET.parse(file)
    rootElement = xml_file.getroot()
    for element in rootElement.iter('data'):
        element.tag = 'revue'
    for element in rootElement.iter('row'):
        element.tag = 'rubrique'
    item = rootElement.find("rubrique")
    for element in item.iter('rubrique'):
        element.tag = 'manchette'
    xml_file.write(file)


In [8]:
for i in list_txt_files:
    os.remove(i)

In [67]:
for file in list_xml_files: 
    with open(file, 'r') as f:
        content = f.read()
    soup = bs.BeautifulSoup(content,'xml')
    for tag in soup.find_all("langue"):
        tag['lan'] = tag.get_text()
    for tag in soup.find_all("type"):
        tag.parent['typ'] = tag.get_text()
    with open(file, 'w') as f:
        f.write(soup.prettify())

-----

<?xml version="1.0" encoding="utf-8"?>